In [4]:
# import packages
%config Completer.use_jedi = False

import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import os

In [2]:
age = pd.read_csv('../demo.csv')

In [5]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_*_RV/ETLearning_*.csv'

db = pd.DataFrame()

for sub in glob(glober):
    
    try:
        df = pd.read_csv(sub)
        df['sub'] = sub.split('_')[2]
        if df.shape[0] == 70:
            db = pd.concat([db, df], axis = 0)
            #db = db.append(df)#[df.trialNum<36])
    except:
        print(sub)
        print('error')

#db['rating'] = db['rating'].replace(0, np.nan)
db = db.sort_values(by=['sub','trialNum'])
print('number of subject: ', len(db['sub'].unique()))

number of subject:  98


In [6]:
db['sub'] = db['sub'].astype('int')
db = db.merge(age, left_on='sub', right_on='sub')
db = db.sort_values('sub')
print('Valid subjects: ', len(np.unique(db['sub'])))

Valid subjects:  70


In [7]:
x = db[db['RT']==-1].groupby('sub').count()['rating']
x = x[x>5].reset_index()
print(x)
db = db[~db['sub'].isin(x['sub'].values)]

   sub  rating
0  102      13


In [8]:
subs = db.drop_duplicates('sub')
subs = subs.drop(['trialNum', 'rectOri', 'rectValue', 'rating', 'RT'],axis = 1).reset_index(drop=True)
subs.head()

,sub,gender,age,moca_score
0,10,M,18,28
1,11,F,43,26
2,13,F,48,28
3,14,F,26,30
4,15,F,58,26


In [10]:
db['rating'] = [np.nan if 0 else x/9 for x in db.rating]
db['rectValue'] = db['rectValue']/6
db['rectOri'] = db['rectOri']/45

In [26]:
# llik function
def llik_td(x, *args):
    # Extract the arguments as they are passed by scipy.optimize.minimize
    alpha, beta = x
    stim, reward, rating  = args
    
    revSim = np.zeros(len(stim))
    revCSa = 0.5
    revCSb = 0.5
    # set intercept and slopes
    for i,(s,r) in enumerate(zip(stim,reward)):
       
        if s==1:      
            pe = r - revCSa   # prediction error
            revCSa = revCSa - alpha*pe
            revSim[i] = revCSa
        if s==0:
            pe = r - revCSb   # prediction error
            revCSb = revCSb - alpha*pe
            revSim[i] = revCSb
        # add intercept and slope
        revSim[i] = revSim[i] 
        
        revSim[i] =  beta*revSim[i]
   
    revPred = revSim
    # Calculate the log-likelihood for normal distribution
    LL = np.sum(scipy.stats.norm.logpdf(rating, revPred))
    # Calculate the negative log-likelihood
    neg_LL = -1*LL
    return neg_LL 

# llik function
def llik_td_in(x, *args):
    # Extract the arguments as they are passed by scipy.optimize.minimize
    alpha, beta, inter = x
    stim, reward, rating  = args
    
    revSim = np.zeros(len(stim))
    revCSa = 0.5
    revCSb = 0.5
    # set intercept and slopes
    for i,(s,t) in enumerate(zip(stim,reward)):
       
        if s==1:      
            pe = t - revCSa   # prediction error
            revCSa = revCSa - alpha*pe
            revSim[i] = revCSa
        if s==0:
            pe = t - revCSb   # prediction error
            revCSb = revCSb - alpha*pe
            revSim[i] = revCSb
        # add intercept and slope
        revSim[i] = revSim[i] 
        
        revSim[i] =  beta*revSim[i] + inter
   
    revPred = revSim
    # Calculate the log-likelihood for normal distribution
    LL = np.sum(scipy.stats.norm.logpdf(rating, revPred))
    # Calculate the negative log-likelihood
    neg_LL = -1*LL
    return neg_LL 

In [37]:
MLL = pd.DataFrame()

sublist = db['sub'].unique()

alpha = 0.5
beta = 0.5
inter = 0

x0 = [alpha, beta]
x1 = [alpha, beta, inter]

for sub in sublist:
    df = db[db['sub'] == sub]
    
    estLog = scipy.optimize.minimize(llik_td, x0, 
                                     args=(df['rectOri'].values, 
                                           df['rectValue'].values,
                                           df['rating']), 
                                     method='L-BFGS-B', bounds=[(0,1),(-2,2)])
    
    estLog_i = scipy.optimize.minimize(llik_td_in, x1,
                                       args=(df['rectOri'].values,
                                             df['rectValue'].values,
                                             df['rating']),
                                       method='L-BFGS-B', bounds=[(0,1),(-2,2),(0,1)])
    
    temp = pd.DataFrame({'sub':sub, 'alphaML0' : estLog.x[0], 'betaML0': estLog.x[1],
                                    'alphaML1' : estLog_i.x[0], 'betaML1': estLog_i.x[1], 'inter': estLog_i.x[2]}, index=[0])
    MLL = pd.concat([MLL, temp])
MLL.head()

,sub,alphaML0,betaML0,alphaML1,betaML1,inter
0,10,0.019357,0.757059,0.170734,0.003383,0.428571
0,11,0.010308,1.242188,0.016371,0.710552,0.267455
0,13,0.006310,1.133345,0.275504,0.000113,0.584508
0,14,0.000000,0.758730,0.000000,0.329180,0.214775
0,15,0.000000,1.260317,0.000000,0.752937,0.253690


In [38]:
pd.merge(MLL, age, left_on='sub', right_on='sub').corr()

,sub,alphaML0,betaML0,alphaML1,betaML1,inter,age,moca_score
sub,1.000000,0.028326,-0.115986,-0.133131,-0.063685,-0.036778,-0.116032,0.070292
alphaML0,0.028326,1.000000,-0.435733,0.182221,-0.154499,-0.165629,-0.246642,0.363547
betaML0,-0.115986,-0.435733,1.000000,-0.112810,0.457450,0.358925,0.314941,-0.252073
alphaML1,-0.133131,0.182221,-0.112810,1.000000,-0.605228,0.547280,-0.185299,-0.018933
betaML1,-0.063685,-0.154499,0.457450,-0.605228,1.000000,-0.663917,0.300287,0.009567
inter,-0.036778,-0.165629,0.358925,0.547280,-0.663917,1.000000,-0.056067,-0.212637
age,-0.116032,-0.246642,0.314941,-0.185299,0.300287,-0.056067,1.000000,-0.299011
moca_score,0.070292,0.363547,-0.252073,-0.018933,0.009567,-0.212637,-0.299011,1.000000
